In [25]:
import json
import os
from PIL import Image, ImageDraw
import requests

# Dataset from: https://github.com/facebookresearch/IMGUR5K-Handwriting-Dataset
class ImgurDataset():
    def __init__(self, dataset_dir):
        self.dataset_dir = dataset_dir
        self._load_annotations()
        self._load_index_ids()

    def _load_annotations(self):
        with open(os.path.join(self.dataset_dir, "dataset_info/imgur5k_annotations.json"), "r") as file:
            data = json.load(file)
        
        self.index_to_ann_map = data.get("index_to_ann_map", {})
        self.ann_id = data.get("ann_id", {})
        self.index_to_image_info = data.get("index_id", {})

    def _load_index_ids(self):
        with open(os.path.join(self.dataset_dir, "dataset_info/train_index_ids.lst"), "r") as file:
            self.index_ids = [line.strip() for line in file]

    def display_image(self, index):
        index_id = self.index_ids[index]
        image_info = self.index_to_image_info.get(index_id)
        if image_info is None:
            print("Image not found for index:", index)
            return
        
        image_path = os.path.join(self.dataset_dir, image_info.get("image_path"))
        if image_path is None:
            print("Image path not found for index:", index)
            return
        if not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            return
        
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        
        ann_ids = self.index_to_ann_map.get(index_id, [])
        for ann_id in ann_ids:
            annotation = self.ann_id.get(ann_id)
            if annotation is None:
                continue
            
            bounding_box = annotation.get("bounding_box")
            word = annotation.get("word")
            if bounding_box is None:
                continue
            
            bounding_box = json.loads(bounding_box)
            # Extract bounding box coordinates
            xc, yc, w, h, a = bounding_box
            # Draw bounding box
            draw.rectangle([(xc - w/2, yc - h/2), (xc + w/2, yc + h/2)], outline="red")
            # Draw text
            draw.text((xc - w/2, yc - h/2 - 20), word, fill="blue")
        
        image.show()

In [1]:
import json
import os
from PIL import Image, ImageDraw
import requests

class ImgurDataset():
    def __init__(self, dataset_dir):
        self.dataset_dir = dataset_dir
        self._load_annotations()
        self._load_index_ids()

    def _load_annotations(self):
        with open(os.path.join(self.dataset_dir, "dataset_info/imgur5k_annotations.json"), "r") as file:
            data = json.load(file)
        
        self.index_to_ann_map = data.get("index_to_ann_map", {})
        self.ann_id = data.get("ann_id", {})
        self.index_to_image_info = data.get("index_id", {})

    def _load_index_ids(self):
        with open(os.path.join(self.dataset_dir, "dataset_info/train_index_ids.lst"), "r") as file:
            self.index_ids = [line.strip() for line in file]

    def _remove_index_id(self, index_id):
        with open(os.path.join(self.dataset_dir, "dataset_info/train_index_ids.lst"), "r") as file:
            lines = file.readlines()
        
        with open(os.path.join(self.dataset_dir, "dataset_info/train_index_ids.lst"), "w") as file:
            for line in lines:
                if line.strip() != index_id:
                    file.write(line)

    def _is_valid_bbox(self, bounding_box):
        if any(coord is None or coord != coord or coord < 0 for coord in bounding_box):
            return False
        return True

    def display_image(self, index):
        index_id = self.index_ids[index]
        image_info = self.index_to_image_info.get(index_id)
        if image_info is None:
            print("Image not found for index:", index)
            self._remove_index_id(index_id)
            return
        
        image_path = os.path.join(self.dataset_dir, image_info.get("image_path"))
        if image_path is None or not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            self._remove_index_id(index_id)
            return
        
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)
        
        ann_ids = self.index_to_ann_map.get(index_id, [])
        for ann_id in ann_ids:
            annotation = self.ann_id.get(ann_id)
            if annotation is None:
                continue
            
            bounding_box = annotation.get("bounding_box")
            word = annotation.get("word")
            if bounding_box is None:
                continue
            
            bounding_box = json.loads(bounding_box)
            if not self._is_valid_bbox(bounding_box):
                self._remove_index_id(index_id)
                print(f"Invalid bounding box coordinates for index {index_id}. Removing from dataset.")
                return

            # Extract bounding box coordinates
            xc, yc, w, h, a = bounding_box
            # Draw bounding box
            draw.rectangle([(xc - w/2, yc - h/2), (xc + w/2, yc + h/2)], outline="red")
            # Draw text
            draw.text((xc - w/2, yc - h/2 - 20), word, fill="blue")
        
        image.show()

In [2]:
# Usage example
dataset = ImgurDataset(dataset_dir=r'C:\Users\IsaacStalley\Documents\GitHub\IMGUR5K-dataset')

In [4]:
for idx in range(len(dataset.index_ids)):
    dataset.display_image(idx)

Image file not found: C:\Users\IsaacStalley\Documents\GitHub\IMGUR5K-dataset\images/sqxthrV.jpg
Invalid bounding box coordinates for index hdbt2Ag. Removing from dataset.
Invalid bounding box coordinates for index zIwYa7z. Removing from dataset.
Invalid bounding box coordinates for index HguCf8u. Removing from dataset.
Image file not found: C:\Users\IsaacStalley\Documents\GitHub\IMGUR5K-dataset\images/MDvEHIM.jpg
Invalid bounding box coordinates for index wtrSqXj. Removing from dataset.
Invalid bounding box coordinates for index hbvmUSQ. Removing from dataset.
Invalid bounding box coordinates for index qbdkqDQ. Removing from dataset.
Invalid bounding box coordinates for index 99sFGww. Removing from dataset.
Invalid bounding box coordinates for index zbheKHy. Removing from dataset.
Invalid bounding box coordinates for index OIuYhD0. Removing from dataset.
Invalid bounding box coordinates for index 0EVcGpe. Removing from dataset.
Invalid bounding box coordinates for index ODjk9uu. Removi

KeyboardInterrupt: 